# Notebook for making input and output datasets for predictive ae

In [3]:
import numpy as np
import keras
import tensorflow as tf
import matplotlib.pyplot as plt

In [ ]:
# !unzip encoder_b_20.zip

Archive:  encoder_b_20.zip
   creating: content/model/encoder_b/
   creating: content/model/encoder_b/assets/
   creating: content/model/encoder_b/variables/
  inflating: content/model/encoder_b/variables/variables.index  
  inflating: content/model/encoder_b/variables/variables.data-00000-of-00001  
  inflating: content/model/encoder_b/saved_model.pb  
  inflating: content/model/encoder_b/keras_metadata.pb  


In [ ]:
# !unzip encoder_u_20.zip

Archive:  encoder_u_20.zip
   creating: content/model/encoder/
  inflating: content/model/encoder/keras_metadata.pb  
  inflating: content/model/encoder/saved_model.pb  
   creating: content/model/encoder/assets/
   creating: content/model/encoder/variables/
  inflating: content/model/encoder/variables/variables.index  
  inflating: content/model/encoder/variables/variables.data-00000-of-00001  


In [ ]:
# !unzip encoder_v_20.zip

Archive:  encoder_v_20.zip
   creating: content/model/encoder/
  inflating: content/model/encoder/keras_metadata.pb  
  inflating: content/model/encoder/saved_model.pb  
   creating: content/model/encoder/assets/
   creating: content/model/encoder/variables/
  inflating: content/model/encoder/variables/variables.index  
  inflating: content/model/encoder/variables/variables.data-00000-of-00001  


In [4]:
data = np.load("../ddpod/starshape_ae_full.npy")

In [5]:
data.shape

(17500, 48, 48, 3)

## Using separate encoders for u v and b

In [ ]:
encoder_uvb_60 = tf.keras.models.load_model("./uv_b_comp/encoder_uvb", custom_objects=None, compile=True, options=None)
encoder_uv_40 = tf.keras.models.load_model("./uv_b_comp/encoder_uv_40", custom_objects=None, compile=True, options=None)
encoder_b_20 = tf.keras.models.load_model("./all_separate_comp/encoder_b_20", custom_objects=None, compile=True, options=None)

Separate the three channels in the AE data

and then pass each of them into each encoder

In [ ]:
input_full = np.delete(data, slice(4,17501,5),axis = 0)
output_full = data[4:17501:5,:,:]

In [ ]:
input_u = input_full[:,:,:,0]
input_v = input_full[:,:,:,1]
input_b = input_full[:,:,:,2]
output_u = output_full[:,:,:,0]
output_v= output_full[:,:,:,1]
central_b= output_full[:,:,:,2]

In [ ]:
#compress u
cae_input_u = encoder_u(input_u)
cae_output_u = encoder_u(output_u)

In [ ]:
# cae_input_u.shape
cae_output_u.shape

TensorShape([3500, 20])

In [ ]:
#compress v
cae_input_v = encoder_v(input_v)
cae_output_v = encoder_v(output_v)

In [ ]:
#compress buildings info
cae_input_b = encoder_b(input_b)
cae_central_b = encoder_b(central_b)

In [ ]:
new_input_u = np.zeros((3500, 20*4))
new_input_v = np.zeros((3500, 20*4))
new_input_b = np.zeros((3500, 20*4))

for i in range(len(new_input_u)):
  fourgrids_u = np.concatenate((cae_input_u[i*4:(i+1)*4]))
  fourgrids_v = np.concatenate((cae_input_v[i*4:(i+1)*4]))
  fourgrids_b = np.concatenate((cae_input_b[i*4:(i+1)*4]))
  new_input_u[i] = fourgrids_u
  new_input_v[i] = fourgrids_v
  new_input_b[i] = fourgrids_b

In [ ]:
new_input_b.shape

(3500, 80)

In [ ]:
cae_central_b.shape

TensorShape([3500, 20])

In [ ]:
cae_input = np.concatenate((new_input_u, new_input_v, new_input_b, cae_central_b), axis = 1)
cae_input.shape

(3500, 260)

In [ ]:
cae_output = np.concatenate((cae_output_u, cae_output_v), axis = 1)

In [ ]:
cae_output.shape

(3500, 40)

In [ ]:
np.save("cae_input_3500.npy", cae_input)
np.save("cae_output_3500.npy", cae_output)

In [ ]:
np.save("sep3500/cae_input_b_20.npy",cae_input_b)
np.save("sep3500/cae_input_u_20.npy",cae_input_u)
np.save("sep3500/cae_input_v_20.npy",cae_input_v)
np.save("sep3500/cae_central_b_20.npy",cae_central_b)
np.save("sep3500/cae_output_u_20.npy",cae_output_u)
np.save("sep3500/cae_output_v_20.npy",cae_output_v)

In [ ]:
!zip -r sep3500.zip ./sep3500/

  adding: sep3500/ (stored 0%)
  adding: sep3500/cae_output_u_20.npy (deflated 7%)
  adding: sep3500/cae_input_b_20.npy (deflated 11%)
  adding: sep3500/cae_central_b_20.npy (deflated 8%)
  adding: sep3500/cae_input_v_20.npy (deflated 7%)
  adding: sep3500/cae_input_u_20.npy (deflated 7%)
  adding: sep3500/cae_output_v_20.npy (deflated 7%)


## Input - uvb *4 grids + buildings central grid / Output - uv central grid

### Input compression - uvb (nSamples, 60*4) 

In [13]:
input_full = np.delete(data, slice(4,17501,5),axis = 0)
output_full = data[4:17501:5,:,:]

In [14]:
nSamples = output_full.shape[0]

In [8]:
encoder = tf.keras.models.load_model("../models/encoder/model/encoder/")
decoder = tf.keras.models.load_model("../models/decoder/model/decoder/")

In [12]:
encoder.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 1000)              121000    
_________________________________________________________________
dropout (Dropout)            (None, 1000)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 1000)              1001000   
_________________________________________________________________
dropout_1 (Dropout)          (None, 1000)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 30)                30030     
Total params: 1,152,030
Trainable params: 1,152,030
Non-trainable params: 0
_________________________________________________________________


In [10]:
cae_uvb = encoder(data)
cae_uvb.shape

ValueError: Input 0 of layer sequential is incompatible with the layer: expected axis -1 of input shape to have value 120 but received input with shape (17500, 48, 48, 3)

### Input compression - b (nSamples, 20)

In [9]:
encoder_b = tf.keras.models.load_model("all_separate_comp/encoder_b_20/model/encoder_b")

### Concatenate (nSamples, 60*4 + 20)

In [ ]:
for i in range(nSamples):
    uvb_

### Ouput compression (nSamples, 40)